<h1>Install JupyterHub and JupyterLab from the ground up</h1>
<h2>JupyterHub and JupyterLab</h2>
JupyterLab enables access to a multiple 'kernels', each one being a given environment for a given language. The most common is a Python environment, for scientific computing usually one managed by the conda package manager.
This guide will set up JupyterHub and JupyterLab seperately from the Python environment. In other words, we treat JupyterHub+JupyterLab as a 'app' or webservice, which will connect to the kernels available on the system. Specifically:
<ul>
    <li>We will create an installation of JupyterHub and JupyterLab using a virtualenv under `/opt` using the system Python.</li>
    <li>We will install conda globally.</li>
    <li>We will create a shared conda environment which can be used by specific group users.</li>
    <li>We will show how users can create their own private conda environments, where they can install whatever they like.</li>
</ul>
The default JupyterHub Authenticator uses PAM to authenticate system users with their username and password. One can <a href="https://jupyterhub.readthedocs.io/en/stable/reference/authenticators.html#authenticators">choose the authenticator</a> that best suits their needs.

<h3>Setup the JupyterHub and JupyterLab in a virtual environment</h3>
First we create a virtual environment under '/opt/jupyterhub'. The '/opt' folder is where apps not belonging to the operating system are commonly installed. Both jupyterlab and jupyterhub will be installed into this virtualenv. Create it with the command:

In [ ]:
python -m venv /opt/jupyterhub/

Now we use pip to install the required Python packages into the new virtual environment. Since we are separating the user interface from the computing kernels, we don't install any Python scientific packages here. The only exception is `ipywidgets` because this is needed to allow connection between interactive tools running in the kernel and the user interface.
<br>
Note that we use `/opt/jupyterhub/bin/python3 -m pip install` each time - this makes sure that the packages are installed to the correct virtual environment.
<br>
Perform the install using the following commands:

In [ ]:
sudo /opt/jupyterhub/bin/python3 -m pip install wheel
sudo /opt/jupyterhub/bin/python3 -m pip install jupyterhub jupyterlab
sudo /opt/jupyterhub/bin/python3 -m pip install ipywidgets

JupyterHub also currently defaults to requiring configurable-http-proxy, which needs nodejs and npm. The versions of these available in Ubuntu therefore need to be installed first (they are a bit old but this is ok for our needs):

In [ ]:
sudo yum install nodejs npm

In [ ]:
sudo npm install -g configurable-http-proxy

<h3>Create the configuration for JupyterHub</h3>
Now we start creating configuration files. To keep everything together, we put all the configuration into the folder created for the virtualenv, under `/opt/jupyterhub/etc`. For each thing needing configuration, we will create a further subfolder and necessary files.

In [ ]:
sudo mkdir -p /opt/jupyterhub/etc/jupyterhub/
cd /opt/jupyterhub/etc/jupyterhub/

Then generate the default configuration file

In [ ]:
sudo /opt/jupyterhub/bin/jupyterhub --generate-config

This will produce the default configuration file `/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py`

You will need to edit the configuration file to make the JupyterLab interface by the default. Set the following configuration option in your `jupyterhub_config.py` file:

In [ ]:
c.Spawner.default_url = '/lab'

<h3>Setup Systemd service</h3>
We will setup JupyterHub to run as a system service using Systemd (which is responsible for managing all services and servers that run on startup in Ubuntu). We will create a service file in a suitable location in the virtualenv folder and then link it to the system services. First create the folder for the service file:

In [ ]:
sudo mkdir -p /opt/jupyterhub/etc/systemd

Then create the following text file using your favourite editor at

In [ ]:
/opt/jupyterhub/etc/systemd/jupyterhub.service

Paste the following service unit definition into the file:

In [ ]:
[Unit]
Description=JupyterHub
After=syslog.target network.target

[Service]
User=root
Environment="PATH=/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/jupyterhub/bin"
ExecStart=/opt/jupyterhub/bin/jupyterhub -f /opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py

[Install]
WantedBy=multi-user.target

This sets up the environment to use the virtual environment we created, tells Systemd how to start jupyterhub using the configuration file we created, specifies that jupyterhub will be started as the `root` user (needed so that it can start jupyter on behalf of other logged in users), and specifies that jupyterhub should start on boot after the network is enabled.

In [ ]:
# Make systemd aware of our service file. First we symlink our file into systemd's directory
sudo ln -s /opt/jupyterhub/etc/systemd/jupyterhub.service /etc/systemd/system/jupyterhub.service

# Tell systemd to reload its configuration files
sudo systemctl daemon-reload

# Enable the service
sudo systemctl start jupyterhub.service

# Check the service status
sudo systemctl status jupyterhub.service

<h3>Access the UI</h3>

You should now be already be able to access jupyterhub using `<your servers ip>:8000` (assuming you haven't already set up a firewall or something). However, when you log in the jupyter notebooks will be trying to use the Python virtualenv that was created to install JupyterHub, this is not what we want.

In [ ]:
# Open the tunnel
ssh -L 8000:localhost:8000 admin@<your servers ip>

<h2>Conda environments</h2>
<h3>Install conda for the whole system</h3>

We will use `conda` to manage Python environments, use the following <a href="https://www.anaconda.com/products/distribution#linux">link</a> to get the sh file.

In [ ]:
# As root
wget https://repo.anaconda.com/archive/Anaconda3-2022.10-Linux-x86_64.sh -P /tmp

# Make it executable
chmod a+x /tmp/Anaconda3-2022.10-Linux-x86_64.sh

/tmp/Anaconda3-2022.10-Linux-x86_64.sh -u -b -p /opt/anaconda3

# Make conda more easily available to users by symlinking the conda shell setup script to the profile 'drop in' folder so that it gets run on login
sudo ln -s /opt/anaconda3/etc/profile.d/conda.sh /etc/profile.d/conda.sh

<h3>Multi-user Anaconda installation on Linux</h3>

In [ ]:
# Add new group of users
sudo groupadd developers

# Change the owner group on anaconda directory
sudo chgrp -R developers /opt/anaconda3

# Change the owner group on jupyterhub directory
sudo chgrp -R developers /opt/jupyterhub

sudo chmod 770 -R /opt/anaconda3

sudo chmod 770 -R /opt/jupyterhub

# Create the `developer1` user
sudo useradd developer1

# Add the user `developer1` to the `developers`
sudo usermod -aG developers developer1

In [ ]:
# Reboot the system
sudo reboot

<h3>Install a default conda environment for all users</h3>

Then create a conda environment to your liking within that folder. Here we have called it 'python' because it will be the obvious default - call it whatever you like. You can install whatever you like into this environment, but you MUST at least install `ipykernel`

In [ ]:
conda create --prefix /opt/anaconda3/envs/python38 python=3.8 ipykernel

Install into the JupyterHub virtualenv. It will only be visible to the JupyterHub installation we have just created. This is useful to avoid conda environments appearing where they are not expected.

In [ ]:
/opt/anaconda3/envs/python38/bin/python -m ipykernel install --prefix=/opt/jupyterhub/ --name 'python38' --display-name "Python3.8 (default)"

# Overwrite the default python
/opt/anaconda3/envs/python38/bin/python -m ipykernel install --prefix /usr/local/ --name 'python' --display-name "Python (default)"

<h2>Setting up a reverse proxy</h2>

The guide so far results in JupyterHub running on port 8000. It is not generally advisable to run open web services in this way - instead, use a reverse proxy running on standard HTTP/HTTPS ports.

<h3>Using Nginx</h3>

Nginx is a mature and established web server and reverse proxy and is easy to install using `sudo yum install nginx`. Details on using Nginx as a reverse proxy can be found elsewhere. Here, we will only outline the additional steps needed to setup JupyterHub with Nginx and host it at a given URL e.g.` <your-server-ip-or-url>/jupyter`. This could be useful for example if you are running several services or web pages on the same server.

To achieve this needs a few tweaks to both the JupyterHub configuration and the Nginx config. First, edit the configuration file `/opt/jupyterhub/etc/jupyterhub/jupyterhub_config.py` and add the line:

In [ ]:
c.JupyterHub.bind_url = 'http://:8000/jupyter'

where `/jupyter` will be the relative URL of the JupyterHub.

Now Nginx must be configured with a to pass all traffic from `/jupyter` to the the local address `127.0.0.1:8000`. Add the following snippet to your nginx configuration file (e.g. /etc/nginx/sites-available/default).

In [ ]:
location /jupyter/ {
    # NOTE important to also set base url of jupyterhub to /jupyter in its config
    proxy_pass http://127.0.0.1:8000;

    proxy_redirect   off;
    proxy_set_header X-Real-IP $remote_addr;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
    proxy_set_header X-Forwarded-Proto $scheme;

    # websocket headers
    proxy_set_header Upgrade $http_upgrade;
    proxy_set_header Connection $connection_upgrade;
}

In [ ]:
map $http_upgrade $connection_upgrade {
        default upgrade;
        '' close;
    }

Nginx will not run if there are errors in the configuration, check your configuration using:

In [ ]:
nginx -t

In [ ]:
# Restart nginx
sudo systemctl restart nginx.service